<a href="https://colab.research.google.com/github/shishiradk/GraphLangchain/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Build a Question Answering application over a Graph Database

In [42]:
!pip install neo4j langchain langchain_community langchain_groq python-dotenv ipykernel

In [43]:
NEO4J_URI="neo4j+s://f10aee37.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="qM-9V4fgHyrHEKyeNr6AmMapupMPzAXBllJqdGNBGNg"

In [44]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [45]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD,refresh_schema=False)
graph

In [46]:
## Dataset Moview
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [47]:
moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n\n"

In [48]:
graph._driver.session().run(moview_query)

In [49]:
graph.refresh_schema()
print(graph.schema)

ValueError: Query object is only supported for session.run

In [50]:
# Manually retrieve schema information
schema_query = """
CALL apoc.meta.graph()
YIELD nodes, relationships
RETURN nodes, relationships
"""

with graph._driver.session() as session:
    result = session.run(schema_query).single()
    nodes = result["nodes"]
    relationships = result["relationships"]

# Format the schema information
node_properties = []
for node in nodes:
    label = node['labels'][0] if node['labels'] else 'Unknown'
    properties = ', '.join(node['properties']) if node['properties'] else 'None'
    node_properties.append(f"({label} {{ {properties} }})")

relationships_schema = []
for rel in relationships:
    start_node = rel['start'][0] if rel['start'] else 'Unknown'
    end_node = rel['end'][0] if rel['end'] else 'Unknown'
    type = rel['type']
    properties = ', '.join(rel['properties']) if rel['properties'] else 'None'
    relationships_schema.append(f"({start_node})-[:{type} {{ {properties} }}]->({end_node})")

# Update the graph schema attribute (this might not be the intended way, but a workaround)
# Note: Directly setting graph.schema might not fully integrate with Langchain's internal
# schema handling. This is a potential workaround if refresh_schema is not working.
# A better approach might be to re-initialize the Neo4jGraph object if possible and safe.
# For now, let's try updating the schema attribute directly.
graph.schema = "\n".join(node_properties) + "\n" + "\n".join(relationships_schema)

print("Schema updated:")
print(graph.schema)

Schema updated:
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown { None })
(Unknown)-[:DIRECTED { None }]->(Unknown)
(Unknown)-[:IN_GENRE { None }]->(Unknown)
(Unknown)-[:FRIEND { None }]->(Unknown)
(Unknown)-[:LIKES { None }]->(Unknown)
(Unknown)-[:ACTED_IN { None }]->(Unknown)
(Unknown)-[:POSTED { None }]->(Unknown)


In [51]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [52]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True)
chain

In [ ]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response

In [ ]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response

In [ ]:
response=chain.invoke({"query":"How many artists are there?"})
response

In [ ]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response